In [2]:
from loader import *
import util

In [3]:
# Setup plotting
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
# Set Matplotlib defaults
plt.rc('figure', autolayout=True)
plt.rc('axes', labelweight='bold', labelsize='large',
       titleweight='bold', titlesize=18, titlepad=10)

In [4]:
# Various imports
import pandas as pd
import numpy as np
import random

from tensorflow import keras
from keras import layers
from keras import callbacks
from keras import optimizers

In [5]:
TRAIN_AUTOENCODER = 0 
TRAIN_NET = 1

TOTAL_AE = 250000
TOTAL_MLP = 750000

BS_AE = 20
BS_MLP = 50
EPOCHS_AE = 50 
EPOCHS_MLP = 201 
RATE_AE = 0.005
DECAY_AE = 0.98
RATE_MLP = 0.005
DECAY_MLP = 0.98

BIAS = 0.15

N_INPUT = 769 
ENCODING_1 = 600 
ENCODING_2 = 400 
ENCODING_3 = 200
ENCODING_4 = 100

HIDDEN_1 = 200
HIDDEN_2 = 400 
HIDDEN_3 = 200
HIDDEN_4 = 100 
N_OUT = 2

VOLUME_SIZE = 25000

export_path = 'net/exports'

#Get the data from the game files
validation_test, validation_test_l = getTest(N_INPUT, 40, 44)
whiteWins, blackWins = getTrain(N_INPUT, TOTAL_MLP, VOLUME_SIZE)

Loading batch number 0
Loading batch number 1
Loading batch number 2
Loading batch number 3
Loading batch number 4
Loading batch number 5
Loading batch number 6
Loading batch number 7
Loading batch number 8
Loading batch number 9
Loading batch number 10
Loading batch number 11
Loading batch number 12
Loading batch number 13
Loading batch number 14
Loading batch number 15
Loading batch number 16
Loading batch number 17
Loading batch number 18
Loading batch number 19
Loading batch number 20
Loading batch number 21
Loading batch number 22
Loading batch number 23
Loading batch number 24
Loading batch number 25
Loading batch number 26
Loading batch number 27
Loading batch number 28
Loading batch number 29


In [6]:
# Creating the siamese autoencoders. SHAPE IS OFF, only position and player to move, not other data
input_shape = blackWins[0].shape
print("Input Shape:", input_shape)

# Layer 1
Pos2Vec = keras.Sequential([
    # layers.Dense(769, activation='relu', input_shape=input_shape),
    layers.Dense(600, activation='relu', input_shape=input_shape),
    layers.Dense(769, activation='sigmoid'),
])

Input Shape: (769,)
Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-03-12 12:08:06.629892: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-03-12 12:08:06.630699: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
print("Defining Learning Rate")
sample_size = 10000
# Defining a decaying learning rate
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.005,
    decay_steps= sample_size / 200,
    decay_rate=0.98,
    name=None)
    
# Compiling model
print("Compiling")
Pos2Vec.compile(
    optimizer=keras.optimizers.SGD(learning_rate=lr_schedule),
    loss='mean_squared_error',
)

Defining Learning Rate
Compiling


In [8]:
# whiteWins, blackWins
print("Generating random train")
sample_size = 10000#1000000
test_size= 1000#1000
white_sample = whiteWins[np.random.randint(whiteWins.shape[0], size=sample_size), :]
black_sample = blackWins[np.random.randint(blackWins.shape[0], size=sample_size), :]

white_test = whiteWins[np.random.randint(whiteWins.shape[0], size=test_size), :]
black_test = blackWins[np.random.randint(blackWins.shape[0], size=test_size), :]

autoencoder_train = np.concatenate([white_sample, black_sample])
autoencoder_test = np.concatenate([white_test, black_test])

early_stopping = keras.callbacks.EarlyStopping(
    patience=500,
    min_delta=0.0001,
    restore_best_weights=True,
)

print("Training")
history = Pos2Vec.fit(
    autoencoder_train, autoencoder_train,
    validation_data=(autoencoder_test, autoencoder_test),
    batch_size=None,
    epochs=10, #200
    callbacks=[early_stopping],
    verbose=1,
)

Generating random sample
Training
Epoch 1/10


2022-03-12 12:08:07.449748: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-03-12 12:08:07.592425: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


624/625 [============================>.] - ETA: 0s - loss: 0.2497

2022-03-12 12:08:11.196929: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


625/625 [==============================] - 4s 4ms/step - loss: 0.2497 - val_loss: 0.2491
Epoch 2/10
625/625 [==============================] - 2s 4ms/step - loss: 0.2487 - val_loss: 0.2482
Epoch 3/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2479 - val_loss: 0.2475
Epoch 4/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2473 - val_loss: 0.2470
Epoch 5/10
625/625 [==============================] - 2s 4ms/step - loss: 0.2468 - val_loss: 0.2465
Epoch 6/10
625/625 [==============================] - 2s 4ms/step - loss: 0.2464 - val_loss: 0.2462
Epoch 7/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2461 - val_loss: 0.2459
Epoch 8/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2459 - val_loss: 0.2457
Epoch 9/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2457 - val_loss: 0.2456
Epoch 10/10
625/625 [==============================] - 2s 4ms/step - loss: 0.2455 - val_loss: 0.2455


In [9]:
# Training the 400 node layer
layer_1 = Pos2Vec.layers[0] # 769 - 600nodes
layer_2 = Pos2Vec.layers[1] # 600 - 769nodes

layer_1.trainable = False
layer_2.trainable = False

Pos2Vec_2 = keras.Sequential([
    # layers.Dense(769, activation='relu', input_shape=input_shape),
    layer_1,
    layers.Dense(400, activation='relu'),
    layers.Dense(600, activation='relu'),
    layer_2,
])

In [10]:
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    0.005,
    decay_steps=100000,
    decay_rate=0.98)

decay_optimizer = keras.optimizers.SGD(learning_rate=lr_schedule)

In [11]:
# Compiling model
Pos2Vec_2.compile(
    optimizer=decay_optimizer,
    loss='mean_squared_error',
)

In [12]:
print("Training")
history = Pos2Vec_2.fit(
    autoencoder_train, autoencoder_train,
    validation_data=(autoencoder_test, autoencoder_test),
    batch_size=None,
    epochs=10, #200
    callbacks=[early_stopping],
    verbose=1,
)

Training
Epoch 1/10
 19/625 [..............................] - ETA: 3s - loss: 0.2478 

2022-03-12 12:08:34.646220: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


623/625 [============================>.] - ETA: 0s - loss: 0.2477

2022-03-12 12:08:37.333993: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


625/625 [==============================] - 3s 5ms/step - loss: 0.2477 - val_loss: 0.2475
Epoch 2/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2474 - val_loss: 0.2472
Epoch 3/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2470 - val_loss: 0.2469
Epoch 4/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2467 - val_loss: 0.2465
Epoch 5/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2464 - val_loss: 0.2462
Epoch 6/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2460 - val_loss: 0.2458
Epoch 7/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2456 - val_loss: 0.2454
Epoch 8/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2452 - val_loss: 0.2450
Epoch 9/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2448 - val_loss: 0.2446
Epoch 10/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2444 - val_loss: 0.2442


In [13]:
# Training the 400 node layer
layer_3 = Pos2Vec_2.layers[1] # 600 - 400nodes
layer_4 = Pos2Vec_2.layers[2] # 400 - 600nodes

layer_3.trainable = False
layer_4.trainable = False

Pos2Vec_3 = keras.Sequential([
    # layers.Dense(769, activation='relu', input_shape=input_shape),
    layer_1,
    layer_3,
    layers.Dense(200, activation='relu'),
    layers.Dense(400, activation='relu'),
    layer_4,
    layer_2,
])

In [14]:
# Compiling model
Pos2Vec_3.compile(
    optimizer=decay_optimizer,
    loss='mean_squared_error',
)

In [15]:
print("Training")
history = Pos2Vec_3.fit(
    autoencoder_train, autoencoder_train,
    validation_data=(autoencoder_test, autoencoder_test),
    batch_size=None,
    epochs=10, #200
    callbacks=[early_stopping],
    verbose=1,
)

Training
Epoch 1/10
 14/625 [..............................] - ETA: 2s - loss: 0.2475  

2022-03-12 12:09:03.357202: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


618/625 [============================>.] - ETA: 0s - loss: 0.2474

2022-03-12 12:09:06.128678: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


625/625 [==============================] - 3s 5ms/step - loss: 0.2474 - val_loss: 0.2474
Epoch 2/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2473 - val_loss: 0.2473
Epoch 3/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2472 - val_loss: 0.2471
Epoch 4/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2471 - val_loss: 0.2470
Epoch 5/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2470 - val_loss: 0.2469
Epoch 6/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2469 - val_loss: 0.2468
Epoch 7/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2467 - val_loss: 0.2467
Epoch 8/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2466 - val_loss: 0.2465
Epoch 9/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2465 - val_loss: 0.2464
Epoch 10/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2463 - val_loss: 0.2462


In [16]:
# Training the 400 node layer
layer_5 = Pos2Vec_3.layers[2] # 400 - 200nodes
layer_6 = Pos2Vec_3.layers[3] # 200 - 400nodes

layer_5.trainable = False
layer_6.trainable = False

Pos2Vec_4 = keras.Sequential([
    # layers.Dense(769, activation='relu', input_shape=input_shape),
    layer_1,
    layer_3,
    layer_5,
    layers.Dense(100, activation='relu'),
    layers.Dense(200, activation='relu'),
    layer_6,
    layer_4,
    layer_2,
])

In [17]:

# Compiling model
Pos2Vec_4.compile(
    optimizer=decay_optimizer,
    loss='mean_squared_error',
)

In [18]:
print("Training")
history = Pos2Vec_4.fit(
    autoencoder_train, autoencoder_train,
    validation_data=(autoencoder_test, autoencoder_test),
    batch_size=None,
    epochs=10, #200
    callbacks=[early_stopping],
    verbose=1,
)

Training
Epoch 1/10
  1/625 [..............................] - ETA: 3:23 - loss: 0.2474

2022-03-12 12:09:32.472992: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


622/625 [============================>.] - ETA: 0s - loss: 0.2474

2022-03-12 12:09:35.610989: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


625/625 [==============================] - 4s 5ms/step - loss: 0.2474 - val_loss: 0.2474
Epoch 2/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2474 - val_loss: 0.2473
Epoch 3/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2473 - val_loss: 0.2473
Epoch 4/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2473 - val_loss: 0.2473
Epoch 5/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2472 - val_loss: 0.2472
Epoch 6/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2472 - val_loss: 0.2472
Epoch 7/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2472 - val_loss: 0.2471
Epoch 8/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2471 - val_loss: 0.2471
Epoch 9/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2471 - val_loss: 0.2471
Epoch 10/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2470 - val_loss: 0.2470


In [19]:
# Training the 400 node layer
layer_7 = Pos2Vec_4.layers[3] # 400 - 200nodes
layer_8 = Pos2Vec_4.layers[4] # 200 - 400nodes

layer_7.trainable = False
layer_8.trainable = False

Pos2Vec_A = keras.Sequential([
    # layers.Dense(769, activation='relu', input_shape=input_shape),
    layer_1,
    layer_3,
    layer_5,
    layer_7,
])

In [ ]:
import pydotplus
from keras.utils.vis_utils import plot_model
from keras.layers import concatenate

pydot = pydotplus

# Duplicating siamese Pos2Vec with tied weights
la0, la1, la2, la3 = Pos2Vec_A.layers
Pos2Vec_B_in = keras.layers.Input(shape=(769,))
lc0 = la0(Pos2Vec_B_in)
lc1 = la1(lc0)
lc2 = la2(lc1)
lc2 = la3(lc2)

Pos2Vec_B = keras.Model(inputs=Pos2Vec_B_in, outputs=lc2)

# Creating DeepChess layers to compare pos2vec
twin_p2v_in = concatenate([Pos2Vec_A.output, Pos2Vec_B.output])
l0 = layers.Dense(400, activation="relu")(twin_p2v_in)
l1 = layers.Dense(200, activation="relu")(l0)
l2 = layers.Dense(100, activation="relu")(l1)
deepchess_out = layers.Dense(2, activation="relu")(l2)

DeepChess = keras.Model(
    inputs=[Pos2Vec_A.input, Pos2Vec_B.input], 
    outputs=[deepchess_out])

plot_model(DeepChess,
            show_shapes=True)

In [138]:
# whiteWins, blackWins
print("Generating random sample")
train_size = 10000#1000000
test_size= 1000#100000

## Generating training data
# sampling white wins and losses (black wins)
white_w_train = whiteWins[np.random.randint(whiteWins.shape[0], size=train_size), :]
white_l_train = blackWins[np.random.randint(blackWins.shape[0], size=train_size), :]

# Creating (W, L) or (L, W) pairs
DeepChess_in_A = np.concatenate((white_w_train[:train_size // 2], white_l_train[:train_size // 2]))
DeepChess_in_B = np.concatenate((white_l_train[train_size // 2:], white_w_train[train_size // 2:]))

# Creating (1, 0) or (0, 1) pairs corresponding to input
DeepChess_out = np.array([(1,0)] * (train_size // 2) + 
                            [(0,1)] * (train_size // 2))


## Generating testing data
# test white wins and losses (black wins)
white_w_test= whiteWins[np.random.randint(whiteWins.shape[0], size=test_size), :]
white_l_test= blackWins[np.random.randint(blackWins.shape[0], size=test_size), :]

# Creating (W, L) or (L, W) pairs
DeepChess_test_in_A = np.concatenate((white_w_test[:test_size // 2], white_l_test[:test_size // 2]))
DeepChess_test_in_B = np.concatenate((white_l_test[test_size // 2:], white_w_test[test_size // 2:]))

# Creating (1, 0) or (0, 1) pairs corresponding to input
DeepChess_test_out = np.array([(1,0)] * (test_size // 2) + 
                            [(0,1)] * (test_size // 2))

Generating random sample


In [139]:
# Compiling model
DeepChess.compile(
    optimizer=decay_optimizer,
    loss='mean_squared_error',
)

In [140]:
lr_schedule2 = keras.optimizers.schedules.ExponentialDecay(
    0.01,
    decay_steps=100000,
    decay_rate=0.99)

decay_optimizer2 = keras.optimizers.SGD(learning_rate=lr_schedule2)

early_stopping = keras.callbacks.EarlyStopping(
    patience=500,
    min_delta=0.0001,
    restore_best_weights=True,
)

print("Training")
history = DeepChess.fit(
    x=[DeepChess_in_A, DeepChess_in_B], y=DeepChess_out,
    validation_split=0.1,
    batch_size=None,
    epochs=100, #1000
    callbacks=[early_stopping],
    use_multiprocessing=True,
    verbose=1,
)

Training
Epoch 1/100
  1/282 [..............................] - ETA: 1:13 - loss: 0.4553

2022-03-12 19:23:43.937288: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


282/282 [==============================] - ETA: 0s - loss: 0.2693

2022-03-12 19:23:46.065445: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


282/282 [==============================] - 2s 8ms/step - loss: 0.2693 - val_loss: 0.3143
Epoch 2/100
282/282 [==============================] - 2s 7ms/step - loss: 0.2481 - val_loss: 0.3097
Epoch 3/100
282/282 [==============================] - 2s 7ms/step - loss: 0.2478 - val_loss: 0.3128
Epoch 4/100
282/282 [==============================] - 2s 7ms/step - loss: 0.2475 - val_loss: 0.3073
Epoch 5/100
282/282 [==============================] - 2s 7ms/step - loss: 0.2472 - val_loss: 0.2929
Epoch 6/100
282/282 [==============================] - 2s 7ms/step - loss: 0.2471 - val_loss: 0.3118
Epoch 7/100
282/282 [==============================] - 2s 7ms/step - loss: 0.2469 - val_loss: 0.2930
Epoch 8/100
282/282 [==============================] - 2s 7ms/step - loss: 0.2467 - val_loss: 0.2982
Epoch 9/100
282/282 [==============================] - 2s 7ms/step - loss: 0.2466 - val_loss: 0.3179
Epoch 10/100
282/282 [==============================] - 2s 7ms/step - loss: 0.2464 - val_loss: 0.3140
E